# **응급상황 자동 인식 및 응급실 연계 서비스**
# **단계4 : 통합 - pipeline**

## **1.환경설정**

### (1) 경로 설정

구글 드라이브 연결

#### 1) 구글 드라이브 폴더 생성
* 새 폴더(project6_2)를 생성하고
* 제공 받은 파일을 업로드

#### 2) 구글 드라이브 연결

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
path = ''

### (2) 라이브러리

#### 1) 필요한 라이브러리 설치

* requirements.txt 파일의 [경로 복사]를 한 후,
* 아래 경로에 붙여 넣기

In [ ]:
# 경로 :
# 경로가 다른 경우 아래 코드의 경로 부분을 수정하세요.

!pip install -r requirements.txt

#### 2) 라이브러리 로딩

In [ ]:
#필요한 라이브러리 설치 및 불러우기
import os
import requests
import xml.etree.ElementTree as ET
import pandas as pd
import openai
from openai import OpenAI
import json
import sys
import torch
sys.path.append(path)

from transformers import AutoTokenizer, AutoModelForSequenceClassification


# 조에서 생성한 모듈 불러오기 -------------
import emergency as my

## **2. 단위 테스트**

* 세부사항 : 아래 단계별로 데이터가 순차적으로 처리되도록 단위 테스트를 진행합니다.

### (1) open ai key 등록

In [ ]:
my.load_key(path)

### (2) audio to text

In [ ]:
audio_path = path + '/audio/'

In [ ]:
from emergency import audio2text
df = my.audio2text(audio_path)
df

,Audio,Transcribed
0,audio1.mp3,지금 아빠가 넘어졌어요. 머리에서 피가 나는데 숨은 쉬고 있어요. 지금 막 일어났어...
1,audio3.mp3,동생이 콩 가지고 놀다가 코에 들어가서 한쪽 코가 막혔어요. 아무리 빼보려 해도 안...
2,audio2.mp3,119죠. 제가 지금 열이 열이 올랐어요. 몇 도냐면은 38도 정도 돼요. 머리가 ...
3,audio5.mp3,화장실에서 미끄러워서 엉덩방아를 찍었어요. 그러고 꼬리뼈가 계속 아파요. 점점 아픈...
4,audio4.mp3,"아까 가다가 머리를 박았는데, 처음에는 괜찮다가, 지금 3시간 정도 지났는데, 머리..."


### (3) text summary

In [ ]:
df = my.text2summary(path, df)
df

,Audio,Transcribed,summary
0,audio1.mp3,지금 아빠가 넘어졌어요. 머리에서 피가 나는데 숨은 쉬고 있어요. 지금 막 일어났어...,"{""summary"": ""머리에서 출혈이 나는 상태인데 의식이 있는 상태이며, 어지럽..."
1,audio3.mp3,동생이 콩 가지고 놀다가 코에 들어가서 한쪽 코가 막혔어요. 아무리 빼보려 해도 안...,"{""summary"": ""콩이 코에 갇혀서 코가 막혔음"", ""부위"": ""코"", ""증상..."
2,audio2.mp3,119죠. 제가 지금 열이 열이 올랐어요. 몇 도냐면은 38도 정도 돼요. 머리가 ...,"{""summary"": ""열이 올라가고 머리가 아프며 띵한 증상 발생"", ""부위"": ..."
3,audio5.mp3,화장실에서 미끄러워서 엉덩방아를 찍었어요. 그러고 꼬리뼈가 계속 아파요. 점점 아픈...,"{""summary"": ""미끄러져 엉덩방아를 다쳐 꼬리뼈가 아픔"", ""부위"": ""엉덩..."
4,audio4.mp3,"아까 가다가 머리를 박았는데, 처음에는 괜찮다가, 지금 3시간 정도 지났는데, 머리...","{""summary"": ""머리를 박고 3시간이 지난 상태인데 머리가 어지럽고 속이 메..."


### (4) 응급실 등급분류

In [ ]:
def model_prediction(df):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    text = df['summary'][0]
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    inputs = {key: value.to(device) for key, value in inputs.items()}  # 각 텐서를 GPU로 이동
    # 모델 예측
    with torch.no_grad():
         outputs = model(**inputs)

    # 로짓을 소프트맥스로 변환하여 확률 계산
    logits = outputs.logits
    probabilities = logits.softmax(dim=1)

    # 가장 높은 확률을 가진 클래스 선택
    pred = torch.argmax(probabilities, dim=-1).item()

    return pred

In [ ]:
pred = model_prediction(df)

### (5) 응급실추천

In [ ]:
hospital = pd.read_csv(path + '/응급실 정보.csv')
hospital

,병원이름,주소,응급의료기관 종류,전화번호 1,전화번호 3,위도,경도
0,(의)내경의료재단울산제일병원,울산광역시 남구 남산로354번길 26 (신정동),응급실운영신고기관,052-220-3300,052-220-3334,35.548238,129.307011
1,(의)서일의료재단기장병원,부산광역시 기장군 기장읍 대청로72번길 6,지역응급의료기관,051-723-0171,051-723-2119,35.236029,129.216492
2,(의)성세의료재단 뉴성민병원,"인천광역시 서구 칠천왕로33번길 17 (석남동, 신석로 70(석남1동, 성민병원))",지역응급의료기관,032-726-1000,032-726-1190,37.508994,126.669479
3,(의)영문의료재단다보스병원,"경기도 용인시 처인구 백옥대로1082번길 18, 다보스종합병원 (김량장동)",지역응급의료센터,031-8021-2114,031-8021-2130,37.234641,127.210499
4,(의)효심의료재단용인서울병원,경기도 용인시 처인구 고림로 81 (고림동),지역응급의료기관,031-337-0114,031-336-0119,37.240316,127.214491
...,...,...,...,...,...,...,...
520,효산의료재단안양샘병원,"경기도 안양시 만안구 삼덕로 9 (안양동, 안양샘병원)",지역응급의료센터,031-467-9717,031-467-9119,37.393404,126.924477
521,효산의료재단지샘병원,"경기도 군포시 군포로 591 (당동, (G샘병원)군포샘병원)",지역응급의료센터,031-389-3000,031-389-3119,37.358645,126.947360
522,효성시티병원,부산광역시 해운대구 해운대로 135 (재송동),응급실운영신고기관,051-709-3000,051-709-3119,35.185413,129.121459
523,흑룡의원,인천광역시 옹진군 백령면 백령로 831,응급실운영신고기관,032-837-6873,032-837-3153,37.959524,124.665499


In [ ]:
import io
import pandas as pd
pd.read_excel(path, "")

In [ ]:
df['위도'], df['경도'] = 37.358877 ,127.114863  #튜플로 재선언 해야됨
df

,Audio,Transcribed,summary,위도,경도
0,audio1.mp3,지금 아빠가 넘어졌어요. 머리에서 피가 나는데 숨은 쉬고 있어요. 지금 막 일어났어...,"{""summary"": ""머리에서 출혈이 나는 상태인데 의식이 있는 상태이며, 어지럽...",37.358877,127.114863
1,audio3.mp3,동생이 콩 가지고 놀다가 코에 들어가서 한쪽 코가 막혔어요. 아무리 빼보려 해도 안...,"{""summary"": ""콩이 코에 갇혀서 코가 막혔음"", ""부위"": ""코"", ""증상...",37.358877,127.114863
2,audio2.mp3,119죠. 제가 지금 열이 열이 올랐어요. 몇 도냐면은 38도 정도 돼요. 머리가 ...,"{""summary"": ""열이 올라가고 머리가 아프며 띵한 증상 발생"", ""부위"": ...",37.358877,127.114863
3,audio5.mp3,화장실에서 미끄러워서 엉덩방아를 찍었어요. 그러고 꼬리뼈가 계속 아파요. 점점 아픈...,"{""summary"": ""미끄러져 엉덩방아를 다쳐 꼬리뼈가 아픔"", ""부위"": ""엉덩...",37.358877,127.114863
4,audio4.mp3,"아까 가다가 머리를 박았는데, 처음에는 괜찮다가, 지금 3시간 정도 지났는데, 머리...","{""summary"": ""머리를 박고 3시간이 지난 상태인데 머리가 어지럽고 속이 메...",37.358877,127.114863


In [ ]:
def recom_em(path, df):
    hospital = pd.read_csv(path + '/응급실 정보.csv')
    x_0, x_1 = df['위도'][0], df['경도'][0]
    x = (x_0, x_1)
    x_lat, x_lon = x[0], x[1]
    a = 0.05  # 초기 검색 범위
    filtered_hospitals = None

    # 최소 3개의 병원이 필터링될 때까지 검색 범위 확장
    while filtered_hospitals is None or len(filtered_hospitals) < 3:
        x_lat_min, x_lon_min = x_lat - a, x_lon - a
        x_lat_max, x_lon_max = x_lat + a, x_lon + a

        # 위도와 경도 조건으로 병원 필터링
        lat_em, lon_em = hospital['위도'], hospital['경도']
        filtered_hospitals = hospital[
            (lat_em >= x_lat_min) & (lat_em <= x_lat_max) &
            (lon_em >= x_lon_min) & (lon_em <= x_lon_max)
        ]

        # 범위 확장
        a += 0.05

    # API 호출 함수
    def get_dist(hospital):
        lat_em_filter, lon_em_filter = hospital['위도'], hospital['경도']
        url = "https://naveropenapi.apigw.ntruss.com/map-direction/v1/driving"
        headers = {
            "X-NCP-APIGW-API-KEY-ID": 'hqev5yhpp0',
            "X-NCP-APIGW-API-KEY": '0hKV6LdcBYambAIlHkVtAMuxCuRD6ypCVQrSGjne',
        }
        params = {
            "start": f"{x_lon}, {x_lat}",  # 출발지 (경도, 위도)
            "goal": f"{lon_em_filter}, {lat_em_filter}",  # 목적지 (경도, 위도)
            "option": "trafast"  # 실시간 빠른 길 옵션
        }
        response = requests.get(url, headers=headers, params=params).json()
        dist = response['route']['trafast'][0]['summary']['distance']  # 거리 (미터)
        return (dist, hospital['병원이름'], hospital['전화번호 3'], hospital['전화번호 1'])
        # 전화번호 3가 응급의료시설번호고, 전화번호 1이 병원번호

    # 병렬 처리로 API 요청 가속화
    with ThreadPoolExecutor() as executor:
         results = list(executor.map(get_dist, filtered_hospitals.to_dict('records')))

    # 거리 기준 정렬
    results.sort(key=lambda x: x[0])

    # 가장 가까운 3개의 병원 출력
    for i in range(min(3, len(results))):
        print(results[i])

In [ ]:
from concurrent.futures import ThreadPoolExecutor
recom_em(path, df)

(2778, '분당서울대학교병원', '031-787-3119', '031-787-2114')
(5018, '대진의료재단분당제생병원', '031-779-0119', '031-779-0114')
(6125, '국군수도병원', '031-725-6119', '031-725-6075')


## **3. 파이프라인**

* 세부사항
    * [2. 단계별 테스트] 의 내용을 순차적으로 정리합니다.
        * 데이터 처리 전 준비작업 : 한번 실행하면 되는 영역
            * 키, 데이터로딩
            * 모델/토크나이저 로딩
        * 입력값이 들어 왔을 때 출력값까지 처리되는 영역

In [ ]:
from concurrent.futures import ThreadPoolExecutor
path = ''
audio_path = path + '/audio/'
hospital = pd.read_csv(path + '/응급실 정보.csv')
save_directory = path + "fine_tuned_bert"
# 모델 로드
model = AutoModelForSequenceClassification.from_pretrained(save_directory)
# 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained(save_directory)

In [ ]:
my.load_key(path)
df = my.audio2text(audio_path)
df = my.text2summary(path, df)
pred = my.model_prediction(df)
recom_em(path, df)
if pred < 3 :
   df['위도'], df['경도'] = map(float, input().split())
   my.recom_em(path, df)
else : print("집에서 휴식을 취하세요")